In [6]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import numpy as np

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# 데이터 로딩
def load_data():
    # 시험지 문항정보 로딩
    test_items = pd.read_csv('../exam1.csv')
    
    # 응답 데이터 로딩
    responses = pd.read_csv('../응답_데이터.csv')
    
    # 문항 IRT 데이터 로딩
    item_irt = pd.read_csv('../문항_매개변수.csv')
    
    # 수험자 능력 데이터 로딩
    examinee_abilities = pd.read_csv('../수험생_능력_추정치.csv')
    
    return test_items, responses, item_irt, examinee_abilities


# 데이터 분석
def analyze_data(test_items, responses, item_irt, examinee_abilities):
    results = {
        '응시자 수': len(responses),
        '문항 수': len(test_items),
        '평균 점수': responses['점수'].mean(),
        '난이도 분포': {
            '쉬움': sum(item_irt['난이도 (beta)'] < -1),
            '중간': sum((item_irt['난이도 (beta)'] >= -1) & (item_irt['난이도 (beta)'] <= 1)),
            '어려움': sum(item_irt['난이도 (beta)'] > 1)
        },
        '고변별도 문항': item_irt[item_irt['변별도 (alpha)'] > 1]['문항 번호'].tolist(),
        '평가영역별 문항 수': test_items['평가영역'].value_counts().to_dict(),
        '합격률': (responses['결과'] == 'PASS').mean() * 100
    }
    return results

# 시각화
def create_visualizations(item_irt, field=None):
    plt.figure(figsize=(10, 6))
    
    if field == '난이도':
        plt.hist(item_irt['난이도 (beta)'], bins=10, edgecolor='black')
        plt.title('난이도 분포')
        plt.xlabel('난이도')
        plt.ylabel('문항 수')
    elif field == '변별도':
        plt.bar(item_irt['문항 번호'], item_irt['변별도 (alpha)'])
        plt.title('문항별 변별도')
        plt.xlabel('문항 번호')
        plt.ylabel('변별도')
    else:
        plt.text(0.5, 0.5, '필드를 선택해주세요', ha='center', va='center', fontsize=20)
    
    plt.tight_layout()
    plt.show()

# 보고서 생성
def generate_report(results):
    report = f"""
# 한국어능력시험 문항 파라미터 분석 요약 보고서

## 1. 전체 요약

### 시험 개요
- 응시자 수: {results['응시자 수']}명
- 문항 수: {results['문항 수']}문항
- 평균 점수: {results['평균 점수']:.2f}점 (100점 만점 기준)
- 합격률: {results['합격률']:.2f}%

### 주요 발견사항
- 문항의 난이도 분포: 쉬움 {results['난이도 분포']['쉬움']}문항, 중간 {results['난이도 분포']['중간']}문항, 어려움 {results['난이도 분포']['어려움']}문항
- {len(results['고변별도 문항'])}개의 문항에서 높은 변별도(>1.0)를 보이고 있습니다.
- 평가영역별 문항 구성: {', '.join([f"{k}: {v}문항" for k, v in results['평가영역별 문항 수'].items()])}

## 2. 난이도 및 변별도 분석

아래에서 난이도 또는 변별도를 선택하여 해당 분포를 확인할 수 있습니다.

## 3. 고변별도 문항 목록
- 고변별도 문항 기준: 변별도 > 1.0
- 목록: {', '.join(map(str, results['고변별도 문항']))}

## 4. 개선 권고사항

1. 난이도 분포가 적절한지 검토하고, 필요시 난이도 조정을 고려하세요.
2. 낮은 변별도(<0.5)를 가진 문항들을 식별하고 개선 방안을 모색하십시오.
3. 고변별도 문항들의 특성을 분석하여 다른 문항 개발에 참고하십시오.
4. 평가영역별 문항 수의 균형을 검토하고, 필요시 조정을 고려하세요.
5. 합격률을 고려하여 전반적인 시험 난이도의 적절성을 평가하십시오.

이러한 개선을 통해 한국어능력시험의 품질을 높이고, 응시자의 실제 능력을 더욱 정확하게 평가할 수 있을 것으로 기대됩니다.
    """
    return report

# 인터랙티브 위젯 생성 및 업데이트 함수
def create_interactive_widgets(item_irt):
    field_dropdown = widgets.Dropdown(
        options=['선택하세요', '난이도', '변별도'],
        value='선택하세요',
        description='필드:',
    )

    def on_field_change(change):
        if change['new'] != '선택하세요':
            create_visualizations(item_irt, change['new'])
        else:
            clear_output(wait=True)
            display(field_dropdown)

    field_dropdown.observe(on_field_change, names='value')
    display(field_dropdown)

# 메인 실행 함수
def main():
    test_items, responses, item_irt, examinee_abilities = load_data()
    results = analyze_data(test_items, responses, item_irt, examinee_abilities)
    report = generate_report(results)
    display(Markdown(report))
    create_interactive_widgets(item_irt)

# 주피터 노트북에서 실행
if __name__ == '__main__':
    main()


# 한국어능력시험 문항 파라미터 분석 요약 보고서

## 1. 전체 요약

### 시험 개요
- 응시자 수: 30명
- 문항 수: 50문항
- 평균 점수: 53.48점 (100점 만점 기준)
- 합격률: 56.67%

### 주요 발견사항
- 문항의 난이도 분포: 쉬움 1문항, 중간 49문항, 어려움 0문항
- 5개의 문항에서 높은 변별도(>1.0)를 보이고 있습니다.
- 평가영역별 문항 구성: 문법: 15문항, 어휘: 13문항, 문화: 12문항, 쓰기: 5문항, 읽기: 5문항

## 2. 난이도 및 변별도 분석

아래에서 난이도 또는 변별도를 선택하여 해당 분포를 확인할 수 있습니다.

## 3. 고변별도 문항 목록
- 고변별도 문항 기준: 변별도 > 1.0
- 목록: 11, 16, 24, 43, 45

## 4. 개선 권고사항

1. 난이도 분포가 적절한지 검토하고, 필요시 난이도 조정을 고려하세요.
2. 낮은 변별도(<0.5)를 가진 문항들을 식별하고 개선 방안을 모색하십시오.
3. 고변별도 문항들의 특성을 분석하여 다른 문항 개발에 참고하십시오.
4. 평가영역별 문항 수의 균형을 검토하고, 필요시 조정을 고려하세요.
5. 합격률을 고려하여 전반적인 시험 난이도의 적절성을 평가하십시오.

이러한 개선을 통해 한국어능력시험의 품질을 높이고, 응시자의 실제 능력을 더욱 정확하게 평가할 수 있을 것으로 기대됩니다.
    

NameError: name 'widgets' is not defined